<a href="https://colab.research.google.com/github/ClearSpear/cameramouse/blob/master/convert_pretrained_handtracking_graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mount Google Drive to save files

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
%cd '/content/gdrive/My Drive/Stanford/EE292D/project/colab_pretraineddetection'

/content/gdrive/My Drive/Stanford/EE292D/project/colab_pretraineddetection


## Prepare TF

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
   raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
print(tf.__version__)

Found GPU at: /device:GPU:0
1.15.2


In [ ]:
tf.enable_eager_execution()

## Prepare TF object detection API

In [ ]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 36444 (delta 16), reused 16 (delta 0), pack-reused 36404
Receiving objects: 100% (36444/36444), 520.24 MiB | 18.95 MiB/s, done.
Resolving deltas: 100% (24292/24292), done.
Checking out files: 100% (2553/2553), done.


In [ ]:
!pip install tf_slim

     |████████████████████████████████| 358kB 7.2MB/s 


In [ ]:
%cd /content/gdrive/My Drive/Stanford/EE292D/project/colab_pretraineddetection/models/research/
!protoc object_detection/protos/*.proto --python_out=.

/content/gdrive/My Drive/Stanford/EE292D/project/colab_pretraineddetection/models/research
object_detection/protos/input_reader.proto: warning: Import object_detection/protos/image_resizer.proto but not used.


In [ ]:
import os
os.environ['PYTHONPATH'] += ':/content/gdrive/My Drive/Stanford/EE292D/project/colab/models/research/:/content/gdrive/My Drive/Stanford/EE292D/project/colab/models/research/slim'

In [ ]:
!python setup.py build
!python setup.py install

running build
running build_py
creating build
creating build/lib
creating build/lib/object_detection
copying object_detection/__init__.py -> build/lib/object_detection
copying object_detection/eval_util.py -> build/lib/object_detection
copying object_detection/eval_util_test.py -> build/lib/object_detection
copying object_detection/export_inference_graph.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph_lib.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph_lib_test.py -> build/lib/object_detection
copying object_detection/exporter.py -> build/lib/object_detection
copying object_detection/exporter_test.py -> build/lib/object_detection
copying object_detection/inputs.py -> build/lib/object_detection
copying object_detection/inputs_test.py -> build/lib/object_detection
copying object_detection/model_hparams.py -> build/lib/object_detection
copyi

Test that installation was successful

In [ ]:
%cd /content/gdrive/My Drive/Stanford/EE292D/project/colab_pretraineddetection/models/research/object_detection/builders/
!python model_builder_test.py

/content/gdrive/My Drive/Stanford/EE292D/project/colab_pretraineddetection/models/research/object_detection/builders


## Download the github with the trained handtracking graph

In [ ]:
!git clone https://github.com/victordibia/handtracking.git

fatal: destination path 'handtracking' already exists and is not an empty directory.


## Convert saved model to frozen graph compatible with tflite

In [ ]:
%cd /content/gdrive/My Drive/Stanford/EE292D/project/colab_pretraineddetection
H = 300
W = 300

/content/gdrive/My Drive/Stanford/EE292D/project/colab_pretraineddetection


In [ ]:
!python models/research/object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path=handtracking/model-checkpoint/ssdmobilenetv1/ssd_mobilenet_v1_coco.config \
--trained_checkpoint_prefix=handtracking/model-checkpoint/ssdmobilenetv1/model.ckpt-200002 \
--output_directory=tflite \
--add_postprocessing_op=true

Instructions for updating:
Please use `layer.__call__` method instead.
W0602 23:00:24.094074 140263377983360 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0602 23:00:25.401958 140263377983360 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0602 23:00:25.431360 140263377983360 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0602 23:00:25.461002 140263377983360 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv befo

In [ ]:
gf = tf.GraphDef()   
m_file = open('tflite/tflite_graph.pb','rb')
gf.ParseFromString(m_file.read())

for n in gf.node:
    print( n.name )
    tensor = n.op # save the input and output tensor

In [ ]:
# A generator that provides a representative dataset
def representative_data_gen():
  dataset_list = tf.data.Dataset.list_files('/content/gdrive/My Drive/Stanford/EE292D/project/colab/egohands/*/*')
  for i in range(100):
    image = next(iter(dataset_list))
    image = tf.io.read_file(image)
    image = tf.io.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [H, W])
    image = tf.cast(image / 255., tf.float32)
    image = tf.expand_dims(image, 0)
    yield [image]

input_tensors = ['normalized_input_image_tensor']
output_tensors = ['TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3']

converter = tf.lite.TFLiteConverter.from_frozen_graph('tflite/tflite_graph.pb', input_tensors, output_tensors, input_shapes={'normalized_input_image_tensor':[1,H,W,3]})
converter.allow_custom_ops = True
converter.optimizations = [tf.lite.Optimize.DEFAULT]
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
#converter.inference_output_type = tf.uint8
converter.representative_dataset = representative_data_gen
tflite_model = converter.convert()

with open("handdetection_ssdmobilenetv1.tflite", "wb") as f:
  f.write(tflite_model)